---
# Demo Use Case - Queries and Analytics on Video (Part 3)

The data scientist now wants to identify video frames in which objects were incorrectly detected.

The metadata for the video stream produced by the object detection job is loaded into a Pandas dataframe.
This dataframe is used to allow the data scientist to view any image stored in SDP.
They can also filter, sort, and aggregate the dataframe using methods that data scientists are familar with.

---

### Install dependencies

See [install_dependencies.ipynb](install_dependencies.ipynb).

### How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import IPython
import cv2
import itertools
import numpy as np
import pandas as pd
import json
import base64
import datetime
import time
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets
from pathlib import Path
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl
from matplotlib import pyplot as plt
from copy import copy
import os

imp.reload(video);

### Define Pravega stream parameters

In [ ]:
gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
#stream = 'object-detector-input-video'
stream = 'object-detector-output-video'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway, options=[
        ('grpc.max_receive_message_length', 9*1024*1024),
    ])
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

### Build timestamp index
This is an index from timestamp to begin stream cut, end stream cut, and event pointer.

This indexed video player uses the event pointer to fetch frames.

In [ ]:
imp.reload(video);
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl

In [ ]:
indexed_stream = IndexedStream(pravega_client, scope, stream)

In [ ]:
%%time
indexed_stream.update_index(force_full=False)

In [ ]:
len(indexed_stream.index_df)

In [ ]:
def clean_recognitions(recognitions):
    return ','.join(np.unique([r['title'] for r in recognitions]))

In [ ]:
indexed_stream.index_df['recog'] = indexed_stream.index_df.recognitions.apply(clean_recognitions)

In [ ]:
# First and last index record
indexed_stream.index_df.iloc[[0,-1]].T

### Video Player

In [ ]:
class PravegaVideoPlayer():
    def __init__(self, indexed_stream, figsize=(12,10), 
                 tz='America/Los_Angeles', strftime='%Y-%m-%d %I:%M:%S.%f %p %z'):
        self.indexed_stream = indexed_stream
        self.figsize = figsize
        self.tz = tz
        self.strftime = strftime
        self.fields_exclude_cols = ['image_array', 'timestamp', 'to_stream_cut', 'from_stream_cut', 'event_pointer', 'ssrc', 'frameNumber',
                                   'chunkIndex', 'finalChunkIndex', 'tags', 'hash', 'recognitions']
        
    def show(self, frame_number):
        video_frame = self.indexed_stream.get_single_video_frame_by_index(frame_number)
        timestamp = video_frame['timestamp']
        self.timestamp_label.value = '%s  (%s)' % (timestamp, timestamp.astimezone(self.tz).strftime(self.strftime))
        self.streamcut_widget.value = video_frame['from_stream_cut']
        fields = video_frame.copy()
        for col in self.fields_exclude_cols:
             if col in fields: del fields[col]
        self.fields_widget.value = str(fields.to_dict())
        plt.figure(figsize=self.figsize)
        plt.imshow(opencv_image_to_mpl(video_frame['image_array']));

    def move_to_prev_frame(self):
        self.frame_number_widget.value = self.frame_number_widget.value - 1
        
    def move_to_next_frame(self):
        self.frame_number_widget.value = self.frame_number_widget.value + 1
    
    def interact(self):
        w = interactive(
            self.show, 
            frame_number=widgets.IntSlider(
                description='Frame',
                min=0, 
                max=len(self.indexed_stream.index_df)-1, 
                step=1, 
                value=0,
                style={'description_width': 'initial'}))
        self.widget = w
        self.frame_number_widget = w.children[0]
        self.output_widget = widgets.Output()
        self.timestamp_label = widgets.Text(description='Timestamp', disabled=True)
        self.streamcut_widget = widgets.Text(description='Stream Cut', disabled=True)
        self.fields_widget = widgets.Textarea(description='Fields', disabled=True)
        
        self.play_widget = widgets.Play(
            value=0,
            min=0,
            max=len(self.indexed_stream.index_df)-1,
            step=1,
            interval=200,  # milliseconds between frames when playing
            description="Press play",
            disabled=False
        )
        widgets.jslink((self.play_widget, 'value'), (self.frame_number_widget, 'value'))
        
        prev_button = widgets.Button(description="<")
        prev_button.on_click(lambda b: self.move_to_prev_frame())
        next_button = widgets.Button(description=">")
        next_button.on_click(lambda b: self.move_to_next_frame())
        report_problem_button = widgets.Button(description="Report Problem")
        buttons = (prev_button, next_button, report_problem_button)
        labels = (self.timestamp_label, self.streamcut_widget, self.fields_widget)
        image_widget = w.children[-1]
        
        w.children = (self.frame_number_widget, self.play_widget) + labels + buttons + (image_widget, self.output_widget)
        
        layout = w.layout
        w.layout.display = 'flex'
        w.layout.flex_flow = 'row wrap'
        w.layout.justify_content = 'flex-start'
        w.layout.align_items = 'flex-start'
        self.frame_number_widget.layout.width = '100%'        
        for child in labels:
            child.layout.width = '100%'
        for child in buttons:
            child.layout.width = '10%'
        image_widget.layout.width = '100%'
        self.output_widget.layout.width = '100%'
        
        display(self.widget)

In [ ]:
player = PravegaVideoPlayer(indexed_stream)

In [ ]:
player.interact()

# Player for filtered/sorted stream

In [ ]:
indexed_stream.index_df.camera.value_counts()

In [ ]:
indexed_stream.index_df.recog.value_counts()

In [ ]:
pd.DataFrame(indexed_stream.index_df.groupby(['camera','recog']).size()).unstack().fillna('-')

In [ ]:
df = indexed_stream.index_df.copy()
#df = df[df.camera==7]
#df = df[df.recog!='']
#df = df[df.recog.str.contains('person')]
#df = df[df.recog.str.contains('boat')]
#df = df[df.recog.str.contains('bus,person')]
#df = df[df.recog.str.contains('motorbike')]
#df = df[df.recog.str.contains('train')]
#df = df[df.recog.str.contains('chair')]
df = df[df.recog.str.contains('dog')]
#df = df[df.recog.str.contains('sofa')]
len(df)

In [ ]:
filtered_stream = copy(indexed_stream)
filtered_stream.index_df = df
player = PravegaVideoPlayer(filtered_stream)
player.interact()

---
# Possible Next Steps

1. The data scientist may now wish to refine the object detection model based on insights obtained by visualizing the data.
2. The data scientist will then deploy the model to SDP as a Flink + TensorFlow job.
3. The job will begin inference from the beginning of time or any other position, catch up to the current time, and continue to perform inference on images in near real time.
4. The data scientist can use these notebooks to view the results of inference.

---